In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, accuracy_score, balanced_accuracy_score
from sklearn.ensemble import GradientBoostingClassifier
from collections import Counter
import os
import warnings
import joblib
from sklearn.inspection import permutation_importance
from scipy.stats import chi2_contingency
import matplotlib.cm as cm
import scipy.sparse
import time

# Configurações para visualização
warnings.filterwarnings('ignore')
plt.style.use('fivethirtyeight')
sns.set_palette('viridis')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

# Função para tratamento de erros
def safe_execution(func, error_message="Ocorreu um erro", *args, **kwargs):
    try:
        return func(*args, **kwargs)
    except Exception as e:
        print(f"{error_message}: {e}")
        return None

# Função para calcular o coeficiente de Cramer's V
def cramers_v(x, y):
    """Calcula o coeficiente de Cramer's V entre duas variáveis categóricas."""
    confusion_matrix = pd.crosstab(x, y)
    chi2 = chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    phi2 = chi2 / n
    r, k = confusion_matrix.shape
    phi2corr = max(0, phi2 - ((k-1)*(r-1))/(n-1))
    rcorr = r - ((r-1)**2)/(n-1)
    kcorr = k - ((k-1)**2)/(n-1)
    return np.sqrt(phi2corr / min((kcorr-1), (rcorr-1)))

# Função para detectar outliers usando IQR
def detect_outliers(df, column):
    """Detecta outliers usando o método IQR."""
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    return outliers

# Função para agrupar faixas salariais
def group_salary_ranges(df, salary_column):
    """Agrupa faixas salariais próximas para reduzir o número de classes."""
    salary_mapping = {
        'Menos de R$ 1.000/mês': 'Até R$ 2.000/mês',
        'de R$ 1.001/mês a R$ 2.000/mês': 'Até R$ 2.000/mês',
        'de R$ 101/mês a R$ 2.000/mês': 'Até R$ 2.000/mês',
        'de R$ 2.001/mês a R$ 3.000/mês': 'R$ 2.001/mês a R$ 4.000/mês',
        'de R$ 3.001/mês a R$ 4.000/mês': 'R$ 2.001/mês a R$ 4.000/mês',
        'de R$ 4.001/mês a R$ 6.000/mês': 'R$ 4.001/mês a R$ 8.000/mês',
        'de R$ 6.001/mês a R$ 8.000/mês': 'R$ 4.001/mês a R$ 8.000/mês',
        'de R$ 8.001/mês a R$ 12.000/mês': 'R$ 8.001/mês a R$ 16.000/mês',
        'de R$ 12.001/mês a R$ 16.000/mês': 'R$ 8.001/mês a R$ 16.000/mês',
        'de R$ 16.001/mês a R$ 20.000/mês': 'R$ 16.001/mês a R$ 30.000/mês',
        'de R$ 20.001/mês a R$ 25.000/mês': 'R$ 16.001/mês a R$ 30.000/mês',
        'de R$ 25.001/mês a R$ 30.000/mês': 'R$ 16.001/mês a R$ 30.000/mês',
        'de R$ 30.001/mês a R$ 40.000/mês': 'Acima de R$ 30.000/mês',
        'Acima de R$ 40.001/mês': 'Acima de R$ 30.000/mês'
    }
    
    df['Faixa salarial agrupada'] = df[salary_column].map(salary_mapping)
    
    print("\nDistribuição das faixas salariais agrupadas:")
    print(df['Faixa salarial agrupada'].value_counts())
    
    return df

# Carregamento dos dados
try:
    file_path = '/kaggle/input/dataset-clean/dados_limpos.csv'
    if not os.path.exists(file_path):
        file_path = 'dados_limpos.csv'
    df = pd.read_csv(file_path)
    print(f"Dados carregados com sucesso de: {file_path}")
except Exception as e:
    print(f"Erro ao carregar os dados: {e}")
    raise Exception("Não foi possível carregar o arquivo de dados. Verifique o caminho.")

# Exibindo informações básicas sobre os dados
print("\nInformações sobre os dados:")
print(f"Número de registros: {df.shape[0]}")
print(f"Número de colunas: {df.shape[1]}")
print("\nColunas disponíveis:")
print(df.columns.tolist())

# Análise exploratória básica
print("\nDistribuição da faixa salarial original:")
salary_dist = df['Faixa salarial mensal'].value_counts()
print(salary_dist)

# Visualizando a distribuição salarial original
plt.figure(figsize=(14, 8))
sns.countplot(y=df['Faixa salarial mensal'], order=df['Faixa salarial mensal'].value_counts().index)
plt.title('Distribuição de Faixas Salariais Originais', fontsize=16)
plt.xlabel('Contagem', fontsize=14)
plt.ylabel('Faixa Salarial', fontsize=14)
plt.tight_layout()
plt.savefig('distribuicao_faixas_salariais_originais.png')
plt.close()

# Agrupando faixas salariais para reduzir o número de classes
df = group_salary_ranges(df, 'Faixa salarial mensal')

# Visualizando a distribuição salarial agrupada
plt.figure(figsize=(14, 8))
sns.countplot(y=df['Faixa salarial agrupada'], order=df['Faixa salarial agrupada'].value_counts().index)
plt.title('Distribuição de Faixas Salariais Agrupadas', fontsize=16)
plt.xlabel('Contagem', fontsize=14)
plt.ylabel('Faixa Salarial', fontsize=14)
plt.tight_layout()
plt.savefig('distribuicao_faixas_salariais_agrupadas.png')
plt.close()

# Análise de correlação entre variáveis categóricas e o target
try:
    categorical_cols = ['Nível de ensino alcançado', 'Área de formação acadêmica', 
                       'Tempo de experiência na área de dados', 'Nível de senioridade',
                       'Gênero do profissional', 'Cor/Raça/Etnia', 'Setor de atuação da empresa',
                       'UF onde mora', 'Cargo atual']
    
    corr_with_target = {}
    for col in categorical_cols:
        if col in df.columns:
            corr_with_target[col] = cramers_v(df[col], df['Faixa salarial agrupada'])
    
    corr_with_target = {k: v for k, v in sorted(corr_with_target.items(), key=lambda item: item[1], reverse=True)}
    print("\nCorrelação (Cramer's V) entre variáveis categóricas e faixa salarial agrupada:")
    for col, corr in corr_with_target.items():
        print(f"{col}: {corr:.4f}")
    
    plt.figure(figsize=(12, 8))
    plt.bar(corr_with_target.keys(), corr_with_target.values())
    plt.xticks(rotation=45, ha='right')
    plt.title("Correlação (Cramer's V) com Faixa Salarial Agrupada", fontsize=16)
    plt.ylabel("Coeficiente de Cramer's V", fontsize=14)
    plt.tight_layout()
    plt.savefig('correlacao_variaveis_faixa_salarial.png')
    plt.close()
except Exception as e:
    print(f"Erro ao calcular correlações: {e}")

# Preparação dos dados para o modelo
features = df[categorical_cols]
target = df['Faixa salarial agrupada']

# Codificando o target
le_target = LabelEncoder()
y = le_target.fit_transform(target)

# Mapeamento dos códigos para as classes
target_mapping = dict(zip(range(len(le_target.classes_)), le_target.classes_))
print("\nMapeamento das classes codificadas:")
for code, label in target_mapping.items():
    print(f"{code}: {label}")

# Divisão estratificada em treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    features, y, test_size=0.3, random_state=42, stratify=y
)
print(f"\nDados divididos em: {X_train.shape[0]} amostras de treino e {X_test.shape[0]} amostras de teste")

# Pré-processamento: transformação de variáveis categóricas
categorical_features = features.columns
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

# Aplicar preprocessador ao conjunto de treino
X_train_transformed = preprocessor.fit_transform(X_train)

# Balanceamento de classes usando oversampling manual (mais rápido que SMOTE)
try:
    from sklearn.utils import resample
    
    if scipy.sparse.issparse(X_train_transformed):
        X_train_dense = X_train_transformed.toarray()
    else:
        X_train_dense = X_train_transformed
    
    X_resampled_list = []
    y_resampled_list = []
    
    class_counts = pd.Series(y_train).value_counts()
    majority_size = class_counts.max()
    
    for class_idx in np.unique(y_train):
        class_indices = np.where(y_train == class_idx)[0]
        class_features = X_train_dense[class_indices]
        class_targets = np.array([class_idx] * len(class_indices))
        
        if len(class_indices) < majority_size:
            n_samples = majority_size
            resampled_features, resampled_targets = resample(
                class_features, class_targets, 
                replace=True, 
                n_samples=n_samples, 
                random_state=42
            )
        else:
            resampled_features, resampled_targets = class_features, class_targets
        
        X_resampled_list.append(resampled_features)
        y_resampled_list.append(resampled_targets)
    
    X_train_resampled = np.vstack(X_resampled_list)
    y_train_resampled = np.concatenate(y_resampled_list)
    
    print("Distribuição das classes após o balanceamento:")
    resampled_class_dist = pd.Series(y_train_resampled).value_counts().sort_index()
    for class_idx, count in resampled_class_dist.items():
        print(f"{target_mapping[class_idx]}: {count} ({count/len(y_train_resampled)*100:.2f}%)")
        
except Exception as e:
    print(f"Erro ao realizar balanceamento: {e}")
    if scipy.sparse.issparse(X_train_transformed):
        X_train_resampled = X_train_transformed.toarray()
    else:
        X_train_resampled = X_train_transformed
    y_train_resampled = y_train

# **CORREÇÃO PRINCIPAL: Grid de parâmetros otimizado e mais eficiente**
gb_clf = GradientBoostingClassifier(random_state=42)

# Grid reduzido para ser mais eficiente (de 486 para 36 combinações)
param_grid_reduced = {
    'n_estimators': [100, 200],  # Reduzido de 3 para 2 opções
    'learning_rate': [0.1, 0.05],  # Reduzido de 3 para 2 opções
    'max_depth': [3, 5, 7],  # Mantido 3 opções
    'min_samples_split': [2, 10],  # Reduzido de 3 para 2 opções
    'min_samples_leaf': [1, 4],  # Reduzido de 3 para 2 opções
    'subsample': [0.8]  # Reduzido para 1 opção
}

# Alternativa ainda mais rápida: RandomizedSearchCV
param_dist = {
    'n_estimators': [50, 100, 150, 200],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'max_depth': [3, 4, 5, 6, 7],
    'min_samples_split': [2, 5, 10, 15],
    'min_samples_leaf': [1, 2, 4, 6],
    'subsample': [0.8, 0.9, 1.0]
}

cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)  # Reduzido de 5 para 3 folds

print("\nEscolha o método de otimização:")
print("1. GridSearchCV com grid reduzido (36 combinações, ~5-10 min)")
print("2. RandomizedSearchCV (20 combinações aleatórias, ~3-5 min)")
print("3. Modelo com parâmetros padrão otimizados (instantâneo)")

# Para automatizar, vamos usar RandomizedSearchCV (opção 2)
optimization_choice = 2

if optimization_choice == 1:
    print("\nIniciando GridSearchCV com grid reduzido...")
    start_time = time.time()
    grid_search = GridSearchCV(
        gb_clf, param_grid_reduced, cv=cv, 
        scoring='balanced_accuracy', n_jobs=-1, verbose=1
    )
    grid_search.fit(X_train_resampled, y_train_resampled)
    best_gb = grid_search.best_estimator_
    print(f"Tempo de execução: {time.time() - start_time:.2f} segundos")
    print("Melhores parâmetros:", grid_search.best_params_)
    
elif optimization_choice == 2:
    print("\nIniciando RandomizedSearchCV...")
    start_time = time.time()
    random_search = RandomizedSearchCV(
        gb_clf, param_dist, n_iter=20, cv=cv,
        scoring='balanced_accuracy', n_jobs=-1, verbose=1, random_state=42
    )
    random_search.fit(X_train_resampled, y_train_resampled)
    best_gb = random_search.best_estimator_
    print(f"Tempo de execução: {time.time() - start_time:.2f} segundos")
    print("Melhores parâmetros:", random_search.best_params_)
    
else:
    print("\nUsando parâmetros otimizados pré-definidos...")
    best_gb = GradientBoostingClassifier(
        n_estimators=150,
        learning_rate=0.1,
        max_depth=5,
        min_samples_split=10,
        min_samples_leaf=4,
        subsample=0.8,
        random_state=42
    )
    best_gb.fit(X_train_resampled, y_train_resampled)

# Transformar X_test
X_test_transformed = preprocessor.transform(X_test)
if scipy.sparse.issparse(X_test_transformed):
    X_test_transformed = X_test_transformed.toarray()

# Previsões
y_pred = best_gb.predict(X_test_transformed)

# Avaliação do modelo
print("\nAcurácia no conjunto de teste: {:.4f}".format(accuracy_score(y_test, y_pred)))
print("Acurácia balanceada no conjunto de teste: {:.4f}".format(balanced_accuracy_score(y_test, y_pred)))

# Relatório de classificação
print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred, 
                          target_names=[target_mapping[i] for i in sorted(np.unique(y_test))], 
                          zero_division=0))

# Matriz de confusão
plt.figure(figsize=(16, 14))
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
           xticklabels=[target_mapping[i] for i in sorted(np.unique(y_test))], 
           yticklabels=[target_mapping[i] for i in sorted(np.unique(y_test))])
plt.title('Matriz de Confusão', fontsize=16)
plt.xlabel('Predito', fontsize=14)
plt.ylabel('Real', fontsize=14)
plt.xticks(rotation=90)
plt.yticks(rotation=0)
plt.tight_layout()
plt.savefig('matriz_confusao.png')
plt.close()
print("Matriz de confusão salva como 'matriz_confusao.png'")

# Salvando o modelo treinado
try:
    model_filename = 'modelo_gradient_boosting_disparidade_salarial_otimizado.pkl'
    joblib.dump(best_gb, model_filename)
    print(f"\nModelo salvo como '{model_filename}'")
    
    preprocessor_filename = 'preprocessador_otimizado.pkl'
    joblib.dump(preprocessor, preprocessor_filename)
    print(f"Preprocessador salvo como '{preprocessor_filename}'")
    
    target_mapping_filename = 'target_mapping.pkl'
    joblib.dump(target_mapping, target_mapping_filename)
    print(f"Mapeamento do target salvo como '{target_mapping_filename}'")
except Exception as e:
    print(f"Erro ao salvar o modelo: {e}")

print("\nModelo treinado e avaliado com sucesso!")


Dados carregados com sucesso de: /kaggle/input/dataset-clean/dados_limpos.csv

Informações sobre os dados:
Número de registros: 3300
Número de colunas: 13

Colunas disponíveis:
['id', 'Nível de ensino alcançado', 'Área de formação acadêmica', 'Faixa salarial mensal', 'Tempo de experiência na área de dados', 'Nível de senioridade', 'Gênero do profissional', 'Cor/Raça/Etnia', 'Setor de atuação da empresa', 'UF onde mora', 'Cargo atual', 'Oportunidade de aprendizado', 'Reputação da empresa']

Distribuição da faixa salarial original:
Faixa salarial mensal
de R$ 8.001/mês a R$ 12.000/mês     790
de R$ 4.001/mês a R$ 6.000/mês      635
de R$ 6.001/mês a R$ 8.000/mês      538
de R$ 12.001/mês a R$ 16.000/mês    388
de R$ 3.001/mês a R$ 4.000/mês      303
de R$ 2.001/mês a R$ 3.000/mês      229
de R$ 1.001/mês a R$ 2.000/mês      172
de R$ 16.001/mês a R$ 20.000/mês    113
de R$ 20.001/mês a R$ 25.000/mês     53
de R$ 25.001/mês a R$ 30.000/mês     30
de R$ 30.001/mês a R$ 40.000/mês     20
Me